# Evolving surfaces

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors
import time
from scipy.interpolate import interp1d
from scipy.signal import correlate
from scipy.optimize import curve_fit
import theano
import theano.tensor as tt
import theano.sparse as ts
import pymc3 as pm
import pymc3.distributions.transforms as tr
import exoplanet as exo
from scipy.sparse import csr_matrix, csc_matrix
from emcee.autocorr import integrated_time
import logging
from scipy.interpolate import interp1d

In [ ]:
starry.config.lazy = False
starry.config.quiet = True
np.random.seed(1)

## Generate

Let's create some dummy classes to store the true parameters and the dataset:

In [ ]:
class Truth(object):
    pass


truth = Truth()

In [ ]:
class Data(object):
    pass


data = Data()

Now we'll define the true parameters of the data. We're going to model ``30`` discrete Gaussian spots centered at various latitudes, each rotating at a period given by the linear differential rotation law. Our dataset will mimic one quarter of Kepler data: about ``4500`` points roughly evenly spaced over ``90`` days.

In [ ]:
truth.ydeg = 20  # degree of the Ylm expansion
truth.inc = 60.0  # stellar inclination
truth.prot = 1.129337  # stellar rotation period
truth.alpha = 0.1  # differential shear

truth.nspots = 30  # number of spots
truth.tau_mu = 20 * truth.prot  # average spot timescale
truth.tau_sig = 1.0  # spot timescale std. dev.

data.tmax = 90  # length of timeseries in days
data.tpad = 50.0  # spots can emerge this many days before t = 0
data.npts = 4499  # number of cadences
data.t = np.sort(
    np.linspace(0, data.tmax, data.npts)
    + (1e-3 * data.tmax / data.npts) * np.random.randn(data.npts)
)  # time array
data.ferr = 1e-3  # photometric noise

Given those parameters, let's draw from some fiducial distributions to obtain our spots, their emergence times, and their timescales:

In [ ]:
# Instantiate a starry map
map = starry.Map(truth.ydeg)

# Spot latitude distribution: isotropic
lat = lambda: (np.arccos(2 * np.random.random() - 1) - 0.5 * np.pi) * 180 / np.pi

# Spot longitude distribution: isotropic
lon = lambda: 360.0 * np.random.random()

# Spot size distribution
sigma = lambda: max(0.01, np.exp(-3.5 + 0.4 * np.random.randn()))

# Spot intensity distribution
intensity = lambda: -min(1.0, 10 * np.exp(-3 + 0.5 * np.random.randn()))

# Generate the Ylm coeffs for each spot
truth.y = np.empty((truth.nspots, (truth.ydeg + 1) ** 2))
truth.lat = np.zeros(truth.nspots)
truth.lon = np.zeros(truth.nspots)
truth.sigma = np.zeros(truth.nspots)
truth.intensity = np.zeros(truth.nspots)
for n in tqdm(range(truth.nspots)):
    map.reset()
    truth.lat[n] = lat()
    truth.lon[n] = lon()
    truth.sigma[n] = sigma()
    truth.intensity[n] = intensity()
    map.add_spot(
        lat=truth.lat[n],
        lon=truth.lon[n],
        sigma=truth.sigma[n],
        intensity=truth.intensity[n],
    )
    truth.y[n] = map.amp * map.y
truth.y[:, 0] = 0

# Spot timescales
truth.tau = truth.tau_mu + truth.tau_sig * np.random.randn(truth.nspots)

# Spot emergence times
truth.t0 = np.sort((data.tmax + data.tpad) * np.random.random(truth.nspots) - data.tpad)

# Spot amplitudes as a function of time
truth.a = np.exp(
    -((data.t.reshape(1, -1) - truth.t0.reshape(-1, 1)) ** 2)
    / (2 * truth.tau.reshape(-1, 1) ** 2)
)

Visualize the distributions:

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(12, 10), sharey=True)
ax = ax.flatten()
ax[0].hist(truth.tau)
ax[0].set_xlabel("timescale [days]")

ax[1].hist(truth.t0)
ax[1].set_xlabel("emergence time [days]")

ax[2].hist(truth.lat)
ax[2].set_xlabel("latitude [degrees]")

ax[3].hist(truth.lon)
ax[3].set_xlabel("longitude [degrees]")

ax[4].hist(truth.intensity)
ax[4].set_xlabel("intensity [fractional]")

ax[5].hist(truth.sigma)
ax[5].set_xlabel("size [fractional]");

In [ ]:
plt.imshow(
    truth.a, aspect="auto", extent=(0, data.tmax, truth.nspots, 0), vmin=0, vmax=1
)
plt.colorbar(label="amplitude")
plt.plot(truth.t0, 0.5 + np.arange(truth.nspots), "w|", ms=7.5)
plt.xlim(0, data.tmax)
plt.xlabel("time [days]")
plt.ylabel("spot number");

Here are our actual spots:

In [ ]:
nx = 6
ny = 5
if nx is None or ny is None:
    nx = 1 + int(np.ceil(np.sqrt(truth.nspots)))
    ny = 1
    while ny * nx < truth.nspots:
        ny += 1
fig, ax = plt.subplots(ny, nx, figsize=(12, 5))
ax = ax.flatten()
for axis in ax:
    axis.axis("off")
for k in tqdm(range(truth.nspots)):
    map.reset()
    map[1:, :] = truth.y[k, 1:]
    img = np.pi * map.render(projection="moll", res=100)
    ax[k].imshow(
        img,
        origin="lower",
        extent=(-1, 1, -0.5, 0.5),
        cmap="Greys_r",
        vmin=0.0,
        vmax=1,
    )
    x_el = np.linspace(-1, 1, 1000)
    y_el = 0.5 * np.sqrt(1 - x_el ** 2)
    ax[k].plot(x_el, y_el, "k-", lw=1, clip_on=False)
    ax[k].plot(x_el, -y_el, "k-", lw=1, clip_on=False)

Compute the empirical mean and covariance of the spot distribution. We're going to use this as our **prior** below.

In [ ]:
N = 99999
y = np.empty((N, (truth.ydeg + 1) ** 2 - 1))
for n in tqdm(range(N)):
    map.reset()
    map.add_spot(lat=lat(), lon=lon(), sigma=sigma(), intensity=intensity())
    y[n] = map.amp * map.y[1:]

truth.ymu = np.mean(y, axis=0)
truth.ycov = np.cov(y.T)
truth.ycov[np.diag_indices_from(truth.ycov)] += 1e-12

Draw and visualize a sample:

In [ ]:
map.reset()
map[1:, :] = np.random.multivariate_normal(truth.ymu, truth.ycov)
map.show(projection="moll", colorbar=True)

Now visualize the star as a movie:

In [ ]:
def get_movie(
    t=data.t,
    y=truth.y,
    lats=truth.lat,
    prot=truth.prot,
    alpha=truth.alpha,
    a=truth.a,
    downsamp=10,
    res=300,
    projection="moll",
):

    # Instantiate a map of the right degree
    map = starry.Map(ydeg=np.sqrt(y.shape[1]) - 1)

    # Theano function for rendering one spot
    def _render(y, theta, res):
        """Render the map on a Mollweide grid."""
        # Compute the Cartesian grid
        if projection == "moll":
            xyz = map.ops.compute_moll_grid(res)[-1]
        else:
            xyz = map.ops.compute_rect_grid(res)[-1]

        # Compute the polynomial basis
        pT = map.ops.pT(xyz[0], xyz[1], xyz[2])

        # Rotate the map
        Ry = map.ops.left_project(
            tt.transpose(tt.tile(y, [theta.shape[0], 1])),
            np.array(0.5 * np.pi),
            np.array(0.0),
            theta,
            np.array(0.0),
            np.array(np.inf),
            np.array(0.0),
        )

        # Change basis to polynomials
        A1Ry = ts.dot(map.ops.A1, Ry)

        # Dot the polynomial into the basis
        res = tt.reshape(tt.dot(pT, A1Ry), [res, res, -1])

        # We need the shape to be (nframes, npix, npix)
        return res.dimshuffle(2, 0, 1)

    # Compile the theano function
    with theano.configparser.change_flags(compute_test_value="off"):
        _y = tt.dvector()
        _theta = tt.dvector()
        _res = tt.iscalar()
        render_spot = theano.function([_y, _theta, _res], _render(_y, _theta, _res))

    # Sum the contribution from each spot in the co-rotating frame
    nim = len(t[::downsamp])
    img = np.ones((nim, res, res))
    theta_eq = 2 * np.pi / prot * t
    theta = theta_eq.reshape(1, -1) * (
        1 - alpha * np.sin(np.pi / 180 * lats.reshape(-1, 1)) ** 2
    )
    theta_diff = theta - theta_eq.reshape(1, -1)
    for k in tqdm(range(len(lats))):
        imgk = np.pi * render_spot(y[k], theta_diff[k, ::downsamp], res)
        img += a[k, ::downsamp].reshape(-1, 1, 1) * imgk

    return img

In [ ]:
truth.movie = get_movie()
map.show(image=truth.movie, projection="moll", colorbar=True)

Get the light curve:

In [ ]:
def get_model(
    t=data.t,
    y=truth.y,
    lats=truth.lat,
    prot=truth.prot,
    inc=truth.inc,
    alpha=truth.alpha,
    a=truth.a,
):

    # Instantiate a map of the right degree
    map = starry.Map(ydeg=np.sqrt(y.shape[1]) - 1, inc=inc)

    # Angular phases of each spot
    theta_eq = 360.0 / prot * t
    theta = theta_eq.reshape(1, -1) * (
        1 - alpha * np.sin(np.pi / 180 * lats.reshape(-1, 1)) ** 2
    )

    # Sum the contribution from each spot
    model = np.ones_like(t)
    for k in range(len(lats)):
        model += a[k] * map.design_matrix(theta=theta[k]).dot(y[k])

    return model

In [ ]:
def plot_lc(t, fluxes, styles=None, nrow=5, ncol=3, figsize=(12, 10)):

    fig = plt.figure(figsize=figsize)
    ax_main = plt.subplot2grid((nrow, ncol), (0, 0), colspan=ncol, rowspan=2)
    ax_sub = [
        plt.subplot2grid((nrow, ncol), (2 + i, j))
        for i in range(nrow - 2)
        for j in range(ncol)
    ]
    nsub = len(ax_sub)
    npts = len(t)

    if styles is None:
        styles = [dict() for flux in fluxes]
    for flux, style in zip(fluxes, styles):
        ax_main.plot(t, flux, **style)

        for k, ax in enumerate(ax_sub):

            a = int(k * npts / nsub)
            b = int((k + 1) * npts / nsub)
            ax.plot(t[a:b], flux[a:b], **style)

    ax_main.legend(fontsize=12, loc="upper right")

    for label in ax_main.get_yticklabels() + ax_main.get_xticklabels():
        label.set_fontsize(10)
    for ax in ax_sub:
        for label in ax.get_yticklabels() + ax.get_xticklabels():
            label.set_fontsize(8)
    ax_main.set_ylabel("flux")
    for ax in ax_sub[-ncol:]:
        ax.set_xlabel("time [days]", fontsize=12)
    for ax in ax_sub[::ncol]:
        ax.set_ylabel("flux", fontsize=12)

In [ ]:
truth.flux0 = get_model(alpha=0)
truth.flux = get_model(alpha=truth.alpha)

Orange is without differential rotation; blue is with differential rotation:

In [ ]:
plot_lc(
    data.t,
    [truth.flux0, truth.flux],
    styles=[
        dict(color="C1", lw=1, alpha=0.5, label="solid"),
        dict(color="C0", lw=2, label="diff rot"),
    ],
)

Finally, generate the dataset we'll do inference on:

In [ ]:
data.flux = truth.flux + data.ferr * np.random.randn(len(truth.flux))

In [ ]:
plot_lc(
    data.t,
    [truth.flux, data.flux],
    styles=[
        dict(color="C0", lw=1, alpha=0.5, label="true"),
        dict(color="k", ls="None", marker=".", ms=2, label="observed"),
    ],
)

## Inference

Now we do inference. Here are the data and parameters we'll assume:

In [ ]:
ydeg = 10  # Ylm degree of the inferred map (this is < the true map!)
N = (ydeg + 1) ** 2  # number of coefficients per node
nnodes = 100  # number of nodes in the linear interpolation

# The dataset
t = data.t
flux = data.flux
ferr = data.ferr
npts = data.npts

# Our priors, which we take to be the actual
# mean and covariance of the process that generated
# the light curve. In reality, we'd marginalize over
# these
ymu = truth.ymu[: N - 1]  # mean of the spatial process
ycov = truth.ycov[: N - 1, : N - 1]  # covariance of the spatial process

# Things we'll assume we know exactly
# (in reality, we'd marginalize over these)
inc = truth.inc
prot = truth.prot

Our temporal model will be a linear interpolation of ``60`` spherical harmonic vectors evaluated on equally spaced nodes. Everything is linear, so we can pre-compute the `starry` design matrix and the interpolation matrix:

In [ ]:
# Pre-compute the starry design matrix
map = starry.Map(ydeg, inc=inc)
theta_x = 360.0 / prot * t
X = map.design_matrix(theta=theta_x)[:, 1:]

In [ ]:
# Pre-compute the interpolation matrix (linear)
tnodes = np.linspace(t[0], t[-1], nnodes)
dt = tnodes[1] - tnodes[0]
diags = np.zeros((nnodes, npts))
for k in range(nnodes):
    w = 1 - np.abs(tnodes[k] - t) / dt
    w[w < 0] = 0
    diags[k] = w
I = np.hstack([np.diag(diag) for diag in diags])

# Visualize
plt.figure()
for diag in diags:
    plt.plot(diag)

Our model for the flux is fully linear and given by

$$
\mathbf{m} = 1 + \mathbf{A} \cdot \mathbf{y}
$$

where $\mathbf{A}$ is the full design matrix (see below) and $\mathbf{y}$ is the vector obtained by concatenating all ``60`` spherical harmonic vectors.

Here's the full design matrix:

In [ ]:
# The full design matrix
from scipy.linalg import block_diag

XL = block_diag(*[X for n in range(nnodes)])
A = I.dot(XL)

In [ ]:
f = np.array(A)
f[f == 0] = np.nan
plt.imshow(f, aspect="auto")
plt.colorbar();

We're going to solve the linear L2 problem, so we need a Gaussian prior on $\mathbf{y}$. This will be the product of a spatial prior and a temporal prior. The spatial prior is the Gaussian defined by the mean and covariance of our spot process, computed above:

In [ ]:
Ls = np.array(ycov)
plt.imshow(Ls)
plt.colorbar();

The temporal prior is a Gaussian Process that correlates nearby nodes to enforce that the surface of the star varies smoothly in time. We're using a Squared Exponential kernel with a single hyperparameter, `gptau`. For real data, the proper thing is to marginalize over `gptau`, but for simplicitly we manually tune it until we get a good fit to the data.

In [ ]:
gptau = 1.0
if gptau > 0:
    k = np.arange(nnodes).reshape(1, -1) - np.arange(nnodes).reshape(-1, 1)
    Lt = np.exp(-0.5 * (k * dt / gptau) ** 2)
else:
    Lt = np.eye(nnodes)
plt.imshow(Lt)
plt.colorbar();

The full covariance matrix is the Kronecker product of these two small matrices:

In [ ]:
L = np.kron(Lt, Ls)
L += 1e-12 * np.eye(L.shape[0])

In [ ]:
fig = plt.figure(figsize=(12, 12))

f = np.array(L)
vmax = np.max(np.abs(f))
f /= vmax
f[np.abs(f) < 1e-5] = 0

imm = plt.imshow(np.log10(-f), cmap="Blues", vmin=-5, vmax=0)
cbm = plt.colorbar(shrink=0.65)
cbm.set_ticks([-5, -4, -3, -2, -1, 0])
cbm.set_ticklabels([r"$-10^{%d}$" % n for n in cbm.get_ticks()])
imp = plt.imshow(np.log10(f), cmap="Reds", vmin=-5, vmax=0)
cbp = plt.colorbar(shrink=0.65)
cbp.set_ticks([-5, -4, -3, -2, -1, 0])
cbp.set_ticklabels([r"$10^{%d}$" % n for n in cbm.get_ticks()])

The prior mean is just the tiled mean of the spatial process:

In [ ]:
mu = np.concatenate([ymu for n in range(nnodes)])

We're now ready to do inference! Since everything is linear, this is just a (very large) matrix inverse problem, which we can tackle with ``starry`` in a few tens of seconds:

In [ ]:
%%time
y, y_cov = starry.linalg.solve(A, flux - 1.0, C=ferr**2, mu=mu, L=L)

Our model (and uncertainty) is given by:

In [ ]:
model = 1 + A.dot(y)
model_sig = np.diag(A.dot(y_cov).dot(A.T))

We can verify that it's a good fit to the data:

In [ ]:
plot_lc(
    t,
    [flux, model],
    styles=[
        dict(color="k", ls="None", marker=".", ms=2, label="observed"),
        dict(color="C0", lw=1, alpha=0.5, label="MAP"),
    ],
    nrow=6,
    ncol=3,
    figsize=(12, 12),
)

We can also visualize the maximum a posterioiri (MAP) solution as a movie:

In [ ]:
def get_y(y, time):
    Y = y.reshape(nnodes, N - 1)
    if time < tnodes[0]:
        return Y[0]
    elif time >= tnodes[-1]:
        return Y[-1]
    k = np.argmin(time >= tnodes) - 1
    return ((time - tnodes[k]) * Y[k + 1] + (tnodes[k + 1] - time) * Y[k]) / (
        tnodes[k + 1] - tnodes[k]
    )

In [ ]:
def _get_image(_y):
    map = starry.Map(ydeg, lazy=True)
    map[1:, :] = _y
    return np.pi * map.render(projection="moll", res=300)


with theano.configparser.change_flags(compute_test_value="off"):
    _y = tt.dvector()
    get_image = theano.function([_y], _get_image(_y))


downsamp = 10
nim = len(t[::downsamp])
movie = np.zeros((nim, 300, 300))
for k in tqdm(range(nim)):
    movie[k] = get_image(get_y(y, t[::downsamp][k]))

In [ ]:
map.show(image=movie, projection="moll", colorbar=True)

## Inferring the Differential Rotation Rate

Given any draw from the posterior, we can compute the "movie" of the surface and determine the differential rotation shear implied by the temporal evolution of features. For simplicity, and as an example, let's compute the shear based on the MAP solution. It's easier if we evaluate the "movie" on a rectangular lat-lon grid:

In [ ]:
def _get_image(_y):
    map = starry.Map(ydeg, lazy=True)
    map[1:, :] = _y
    return np.pi * map.render(projection="rect", res=300)


with theano.configparser.change_flags(compute_test_value="off"):
    _y = tt.dvector()
    get_image = theano.function([_y], _get_image(_y))


downsamp = 2
nim = len(t[::downsamp])
img = np.zeros((nim, 300, 300))
for k in tqdm(range(nim)):
    img[k] = get_image(get_y(y, t[::downsamp][k]))

In [ ]:
lat, lon = map.get_latlon_grid(300, projection="rect")

Now, let's obtain an estimate of the shear by computing the cross-correlation of the brightness of the star at a given latitude with the same brightness profile a certain amount of time earlier (for definiteness, we'll assume a lag of ``30`` frames, which is long enough so that shearing has changed the profile but not too long such that the spots have since dissipated; note that the results aren't too sensitive to this choice).

For every latitude and at every frame, we compute the delta-longitude at which the autocorrelation peaks; this is an estimate of how far the feature has rotated over those ``30`` frames. We then average this shift over all frames to obtain the empirical shearing profile, and compare it to the true profile below.

In [ ]:
lag = 30

Z = np.zeros((nim, 300)) * np.nan
for k in range(300):

    if np.abs(lat[k][0]) > 80:
        Z[:, k] = np.nan
        continue

    f = img[:, k, :]
    idx = ~np.isnan(f[0])
    for j in range(lag, nim):
        f0 = f[j - lag][idx]
        fj = f[j][idx]
        if len(fj):
            corr = correlate(np.tile(f0, 2), fj, mode="valid")
            cc = len(corr) - 1 - np.argmax(corr)
            if cc > len(corr) // 2:
                cc -= len(corr) - 1
            Z[j, k] = cc * np.nanmean(np.diff(lon[k]))

Let's visualize the shear as a function of latitude and time (top panell) and averaged over time (bottom panel):

In [ ]:
fig, ax = plt.subplots(2, figsize=(10, 7), sharex=True)

im = ax[0].imshow(
    Z, aspect="auto", extent=(-90, 90, data.tmax, 0), vmin=-20, vmax=20, cmap="RdBu",
)
plt.colorbar(im, ax=ax[0], label="lag [deg]")

delt = t[lag] - t[0]
signal = -truth.alpha * 360.0 / truth.prot * delt * np.sin(lat[:, 0] * np.pi / 180) ** 2
ax[1].plot(lat[:, 0], signal, label="truth", color="k")
ax[1].set_xlabel("latitude [deg]")
ax[1].set_ylabel("lag [deg]")
ax[0].set_ylabel("time [days]")

mean = np.nanmean(Z, axis=0)
mean -= np.nanmax(mean)
mean_err = np.nanstd(Z, axis=0)
mean_err[lat[:, 0] < -inc] = 1e50

mean_err[150:] = mean_err[:150][::-1]

med = np.nanmedian(Z, axis=0)
med -= np.nanmax(med)

ax[1].plot(lat[:, 0], mean, label="mean", color="C0")
ax[1].fill_between(
    lat[:, 0], mean - mean_err, mean + mean_err, color="C0", alpha=0.3,
)

ax[1].plot(lat[:, 0], med, label="median", color="C1")

ax[1].set_xlim(-80, 80)
ax[1].set_ylim(-20, 2)
cb_ = plt.colorbar(im, ax=ax[1])
cb_.ax.set_visible(False)
ax[1].legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0);

Finally, let's actually fit a differential rotation law to the blue curve to get an empirical estimate of the shear:

In [ ]:
def func(x, alpha):
    return -alpha * 360.0 / truth.prot * delt * np.sin(x * np.pi / 180) ** 2


x = np.array(lat[:, 0])
y = mean - np.nanmax(mean)

x = x[~np.isnan(y)]
yerr = mean_err[~np.isnan(y)]
y = y[~np.isnan(y)]

The inferred shear is:

In [ ]:
X = func(x, 1).reshape(-1, 1)
alpha = np.linalg.solve((X.T / yerr ** 2).dot(X), (X.T / yerr ** 2).dot(y))[0]
alpha_err = np.sqrt(1 / (X.T / yerr ** 2).dot(X))[0, 0]

print("{:.3f} +/- {:.3f}".format(alpha, alpha_err))

In [ ]:
plt.figure(figsize=(8, 3))

plt.plot(x, y, color="C0", label="fit")
plt.fill_between(x, y - yerr, y + yerr, alpha=0.3, color="C0")
plt.ylim(-20, 2)
plt.plot(x, func(x, truth.alpha), color="k", label="truth")
plt.xlabel("latitude [deg]")
plt.ylabel("lag [deg]")
plt.legend();

## Inferring the spot timescale

In [ ]:
plt.imshow(
    img[:, 150, :].T, aspect="auto", extent=(0, data.tmax, 0, 360), origin="lower"
);

In [ ]:
img0 = get_movie(projection="rect", downsamp=2)
plt.imshow(
    img0[:, 150, :].T, aspect="auto", extent=(0, data.tmax, 0, 360), origin="lower"
);

In [ ]:
logging.getLogger().setLevel(50)
tau = np.zeros(300)
tau0 = np.zeros(300)
step = t[::downsamp][1] - t[::downsamp][0]
for k in range(300):
    tau[k] = step * integrated_time(img[:, 150, k], quiet=True)
    tau0[k] = step * integrated_time(img0[:, 150, k], quiet=True)

In [ ]:
plt.hist(tau0, histtype="step", label="truth", lw=2, bins=5)
plt.hist(tau, histtype="step", label="MAP", lw=2, bins=5)
plt.legend()
plt.xlabel("autocorrelation time [days]");

In [ ]:
print("True autocorr time: {:.2f} +/- {:.2f} days".format(np.mean(tau0), np.std(tau0)))
print(
    "Inferred autocorr time: {:.2f} +/- {:.2f} days".format(np.mean(tau), np.std(tau))
)